In [37]:
# Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load Fama-French 3 Factors
ff_factors = pd.read_csv("F-F_Research_Data_Factors 3.CSV", skiprows=3, index_col=0)

# Convert index to string and remove leading/trailing whitespace
ff_factors.index = ff_factors.index.astype(str).str.strip()

# drop non-date rows --> keep only rows with YYYYMM format
ff_factors = ff_factors[ff_factors.index.str.match(r'^\d{6}$')]

# Convert  string index to datetime objects
ff_factors.index = pd.to_datetime(ff_factors.index, format="%Y%m")

# Strip whitespace from  column names
ff_factors.columns = ff_factors.columns.str.strip()

# Convert 'Mkt-RF' column to numeric and divide by 100 (to change from percent to decimal)
# 'errors="coerce"' turns any bad data (e.g. text or missing values) into NaN
ff_factors['MKT_RF'] = pd.to_numeric(ff_factors['Mkt-RF'], errors='coerce')

# Convert 'RF' (risk-free rate) to numeric; coerce ensures invalid values become NaN
ff_factors['RF'] = pd.to_numeric(ff_factors['RF'], errors='coerce')

# Keep only the columns we care about, and drop rows where either is NaN
ff_factors = ff_factors[['MKT_RF', 'RF']].dropna()


#  Load Strategy Returns===
strategies = pd.read_csv("ps1_strategies.csv", index_col=0)
strategies.index = strategies.index.astype(str).str.strip()
strategies.index = pd.to_datetime(strategies.index, format="%Y%m")
strategies = strategies.apply(pd.to_numeric, errors='coerce').dropna()

#  Merge DFs
data = strategies.join(ff_factors, how='inner')

#  CAPM Estimation 
def estimate_capm(excess_returns, market_excess):
    """
    Estimate alpha and beta using CAPM using ordinary least squares regression
    """
    X = sm.add_constant(market_excess / 100)  # convert to decimal
    model = sm.OLS(excess_returns / 100, X).fit()  # convert to decimal
    alpha = model.params['const']*100*100 # OOM doesn't make sense so compensating to get back up
    beta = model.params['MKT_RF']
    return alpha, beta, model

# After fee returns
def simulate_after_fee_capm(strategy_name, data, initial_investment=100, mgmt_fee_rate=0.018 / 12, perf_fee_rate=0.2):
    gross_returns = (data[strategy_name] + data['RF']) / 100
    portfolio_value = initial_investment
    high_water_mark = initial_investment

    management_fees = []
    incentive_fees = []
    values = []

    for r in gross_returns:
        start_val = portfolio_value
        gross_val = start_val * (1 + r)

        mgmt_fee = start_val * mgmt_fee_rate
        post_mgmt_val = gross_val - mgmt_fee

        if post_mgmt_val > high_water_mark:
            incentive_fee = perf_fee_rate * (post_mgmt_val - high_water_mark)
            post_fee_val = post_mgmt_val - incentive_fee
            high_water_mark = post_fee_val
        else:
            incentive_fee = 0
            post_fee_val = post_mgmt_val

        portfolio_value = post_fee_val
        values.append(portfolio_value)
        management_fees.append(mgmt_fee)
        incentive_fees.append(incentive_fee)

    values_series = pd.Series(values, index=data.index)
    net_returns = values_series.pct_change().dropna()
    net_excess_returns = net_returns - data['RF'].loc[net_returns.index] / 100
    market_excess = data['MKT_RF'].loc[net_returns.index] / 100

    alpha, beta, model = estimate_capm(net_excess_returns * 100, market_excess * 100)  # convert back to percent

    return {
        'after_fee_alpha': alpha,
        'after_fee_beta': beta,
        'total_fees': sum(management_fees) + sum(incentive_fees),
        'mgmt_fees': sum(management_fees),
        'incentive_fees': sum(incentive_fees),
        'final_value': portfolio_value
    }

# === Run and Print All Results for LB and HB ===
strategies_to_check = ['LB', 'HB']
print("RESULTS\n")

for strat in strategies_to_check:
    print(f"{strat} Strategy")

    # Before-fee CAPM
    alpha_bf, beta_bf, _ = estimate_capm(data[strat], data['MKT_RF'])
    print(f"Before-Fee Alpha: {alpha_bf:.4f}%")
    print(f"Before-Fee Beta:  {beta_bf:.4f}")

    # After-fee CAPM
    result = simulate_after_fee_capm(strat, data)
    print(f"After-Fee Alpha: {result['after_fee_alpha']:.4f}%")
    print(f"After-Fee Beta: {result['after_fee_beta']:.4f}")
    print(f"Total Fees Paid: ${result['total_fees']:.2f}M")
    print(f"Management: ${result['mgmt_fees']:.2f}M")
    print(f"Incentive (): ${result['incentive_fees']:.2f}M")
    print(f"Final Portfolio Value: ${result['final_value']:.2f}M\n")


RESULTS

LB Strategy
Before-Fee Alpha: 45.2938%
Before-Fee Beta:  0.0209
After-Fee Alpha: 20.5619%
After-Fee Beta: 0.0160
Total Fees Paid: $277.11M
Management: $173.20M
Incentive (): $103.91M
Final Portfolio Value: $496.93M

HB Strategy
Before-Fee Alpha: 37.2236%
Before-Fee Beta:  2.9978
After-Fee Alpha: -9.8439%
After-Fee Beta: 2.9302
Total Fees Paid: $7535.25M
Management: $1585.92M
Incentive (): $5949.33M
Final Portfolio Value: $12030.85M



Suppose a client invests $100 million in each strategy. Calculate the total fees paid to
each strategy. Which has higher fees, why?

### Question 3: Fees
3a/3b: Before-fee alphas and betas, after-fee alphas and betas, as well as total fees paid to each strategy shown immedietly. above.

### 3c: *Suppose a client invests $100 million in each strategy. Calculate the total fees paid to each strategy. Which has higher fees, why?*
The HB strategy has much higher fees. This strategy has higher absolute return. Fees are based on profit, not strictly alpha. Thus, even though the strategies have similar alpha this strategy has much higher fees. Additionally, the HB strategy's high beta leads to very large gains in bull markets.

### 3d: Clients value hedge funds because of alpha, but fees are paid based on absolute returns. For instance, in this question we have seen that high betas lead to higher returns and higher fees. Some of the best performing hedge funds in the world have high market betas, why do you think this is the case?
High-beta funds generate larger absolute returns in bull markets which directly increases both management and incentive fees under the given fee structure. Even if the alpha is small, the shear scale of nominal gains drives fee revenue which grows exponeitally in a prolonged bull market. Many top-performing funds lean into beta because it scales more predictably and is easier to monetize than hard-to-source alpha. This incentive structure rewards exposure over skill, so high-beta strategies thrive even when they are in parity with the market.
